<a href="https://colab.research.google.com/github/danielsadoc/RL_jack_car_rental/blob/main/blackwell_jack_rental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Jack's Car Rental — Discounted vs Average Reward (Blackwell Optimality)

This notebook illustrates the **Blackwell optimality theorem** through the Jack’s Car Rental MDP:

- Solve **average-reward MDP** via occupancy measure LP
- Solve **discounted MDP** for several \( \gamma \uparrow 1 \)
- Compare optimal policies \( \pi^*_{\gamma} \) and \( \pi^{\text{avg}} \)
- Identify empirical Blackwell region.

In [ ]:
import math
import numpy as np
from itertools import product
from scipy.optimize import linprog
from scipy.stats import poisson
import matplotlib.pyplot as plt

# Professor's code

## 1. Problem parameters and state space

In [ ]:
MAX_CARS = 20
A_MAX_MOVE = 5
ACTIONS_ALL = np.arange(-A_MAX_MOVE, A_MAX_MOVE+1)

LAMBDA_X1, LAMBDA_Y1 = 3.0, 3.0
LAMBDA_X2, LAMBDA_Y2 = 4.0, 2.0
RENT_REWARD = 10.0
MOVE_COST = 2.0

STATES = [(n1, n2) for n1 in range(MAX_CARS+1) for n2 in range(MAX_CARS+1)]
S_index = {s: i for i, s in enumerate(STATES)}
NUM_STATES = len(STATES)

## 2. Feasible actions and Poisson helpers

In [ ]:
def feasible_actions(n1, n2):
    feas = []
    for a in ACTIONS_ALL:
        if a >= 0:
            if n1 >= a and n2 + a <= MAX_CARS:
                feas.append(a)
        else:
            if n2 >= -a and n1 - a <= MAX_CARS:
                feas.append(a)
    return feas

FEAS_ACTIONS = [feasible_actions(n1, n2) for (n1, n2) in STATES]

def poisson_pmf(lam, k):
    if k < 0:
        return 0.0
    return math.exp(-lam) * (lam**k) / math.factorial(k)

def poisson_cdf(lam, k):
    return sum(poisson_pmf(lam, i) for i in range(k+1))

def poisson_tail_prob(lam, k):
    if k <= 0:
        return 1.0
    return 1.0 - poisson_cdf(lam, k-1)

## 3. Transition and reward model

In [ ]:
RENTAL_CACHE = {}
NEXTCOUNT_CACHE = {}

def rental_probabilities(c, lam_x):
    probs = np.zeros(c+1)
    if c == 0:
        probs[0] = 1.0
        return probs
    for r in range(c):
        probs[r] = poisson_pmf(lam_x, r)
    probs[c] = poisson_tail_prob(lam_x, c)
    return probs / probs.sum()

def next_count_distribution_given_rentals(c, r, lam_y):
    remaining = c - r
    probs = np.zeros(MAX_CARS+1)
    for nprime in range(MAX_CARS):
        y = nprime - remaining
        if y >= 0:
            probs[nprime] = poisson_pmf(lam_y, y)
    tail_needed = MAX_CARS - remaining
    probs[MAX_CARS] = poisson_tail_prob(lam_y, max(tail_needed, 0))
    return probs / probs.sum()

def get_rental_probs(c, lam_x):
    key = (c, lam_x)
    if key not in RENTAL_CACHE:
        RENTAL_CACHE[key] = rental_probabilities(c, lam_x)
    return RENTAL_CACHE[key]

def get_nextcount_probs(c, r, lam_y):
    key = (c, r, lam_y)
    if key not in NEXTCOUNT_CACHE:
        NEXTCOUNT_CACHE[key] = next_count_distribution_given_rentals(c, r, lam_y)
    return NEXTCOUNT_CACHE[key]

def post_move_counts(n1, n2, a):
    c1 = n1 - a
    c2 = n2 + a
    return c1, c2

def expected_rentals_for_lot(c, lam_x):
    probs = get_rental_probs(c, lam_x)
    support = np.arange(len(probs))
    return (support * probs).sum()

def transition_and_reward_for_state_action(n1, n2, a):
    c1, c2 = post_move_counts(n1, n2, a)
    e_r1 = expected_rentals_for_lot(c1, LAMBDA_X1)
    e_r2 = expected_rentals_for_lot(c2, LAMBDA_X2)
    reward = RENT_REWARD * (e_r1 + e_r2) - MOVE_COST * abs(a)
    r1_probs = get_rental_probs(c1, LAMBDA_X1)
    r2_probs = get_rental_probs(c2, LAMBDA_X2)
    n1prime_given_r1 = [get_nextcount_probs(c1, r1, LAMBDA_Y1) for r1 in range(c1+1)]
    n2prime_given_r2 = [get_nextcount_probs(c2, r2, LAMBDA_Y2) for r2 in range(c2+1)]
    P_row = np.zeros(NUM_STATES)
    for r1 in range(c1+1):
        for r2 in range(c2+1):
            p_r = r1_probs[r1] * r2_probs[r2]
            joint = np.outer(n1prime_given_r1[r1], n2prime_given_r2[r2]) * p_r
            for n1p in range(MAX_CARS+1):
                base = n1p * (MAX_CARS+1)
                P_row[base:base + (MAX_CARS+1)] += joint[n1p, :]
    return P_row, reward

## 4. Average-reward LP (occupancy measures)

In [ ]:
pairs = []
R_vec = []
P_dense_rows = []

for s_idx, (n1, n2) in enumerate(STATES):
    for a in FEAS_ACTIONS[s_idx]:
        P_row, R_sa = transition_and_reward_for_state_action(n1, n2, a)
        pairs.append((s_idx, a))
        R_vec.append(R_sa)
        P_dense_rows.append(P_row)

K = len(pairs)
R_vec = np.array(R_vec)
P_dense = np.vstack(P_dense_rows)

A_eq = np.zeros((NUM_STATES+1, K))
b_eq = np.zeros(NUM_STATES+1)
for k, (s_idx, a) in enumerate(pairs):
    A_eq[s_idx, k] += 1.0
    A_eq[:NUM_STATES, k] -= P_dense[k, :]
A_eq[NUM_STATES, :] = 1.0
b_eq[NUM_STATES] = 1.0

res = linprog(-R_vec, A_eq=A_eq, b_eq=b_eq, bounds=[(0,None)]*K, method='highs')
z = res.x
avg_reward = R_vec @ z
print(f"Optimal average reward per day: {avg_reward:.4f}")

pi = {s_idx: {} for s_idx in range(NUM_STATES)}
z_state = np.zeros(NUM_STATES)
for k, (s_idx, a) in enumerate(pairs):
    pi[s_idx][a] = z[k]
    z_state[s_idx] += z[k]
avg_policy_actions = np.zeros(NUM_STATES, dtype=int)
for s_idx in range(NUM_STATES):
    if z_state[s_idx] > 0:
        avg_policy_actions[s_idx] = max(pi[s_idx].items(), key=lambda kv: kv[1])[0]
    else:
        feas = FEAS_ACTIONS[s_idx]
        avg_policy_actions[s_idx] = min(feas, key=abs) if feas else 0

plt.imshow(avg_policy_actions.reshape((MAX_CARS+1, MAX_CARS+1)), origin='lower')
plt.title('Average Reward Policy (argmax z)')
plt.xlabel('N2'); plt.ylabel('N1'); plt.colorbar(); plt.show()

## 5. Discounted LP for a ladder of $\gamma$ values

In [ ]:
def solve_discounted_lp_and_policy(gamma):
    A = np.zeros((K, NUM_STATES))
    b = np.zeros(K)
    for k, (s_idx, a) in enumerate(pairs):
        A[k, :] = gamma * P_dense[k, :]
        A[k, s_idx] -= 1.0
        b[k] = -R_vec[k]
    res = linprog(np.ones(NUM_STATES), A_ub=A, b_ub=b, bounds=[(None,None)]*NUM_STATES, method='highs')
    v = res.x
    policy = np.zeros(NUM_STATES, dtype=int)
    for s_idx, (n1, n2) in enumerate(STATES):
        best_q, best_a = -1e300, 0
        for a in FEAS_ACTIONS[s_idx]:
            k = next(k for k,(ss,aa) in enumerate(pairs) if ss==s_idx and aa==a)
            q = R_vec[k] + gamma * (P_dense[k,:] @ v)
            if q > best_q:
                best_q, best_a = q, a
        policy[s_idx] = best_a
    return policy

gammas = [0.95, 0.97, 0.98, 0.99, 0.995, 0.997, 0.999]
policies_disc = {g: solve_discounted_lp_and_policy(g) for g in gammas}
rates = {g: np.mean(policies_disc[g]==avg_policy_actions) for g in gammas}
for g in gammas:
    print(f"γ={g}: match rate = {rates[g]*100:.1f}%")

## 6. Visualization of Blackwell phenomenon

In [ ]:
cols = len(gammas)+1
plt.figure(figsize=(3*cols,5))
ax = plt.subplot(1,cols,1)
ax.imshow(avg_policy_actions.reshape((MAX_CARS+1, MAX_CARS+1)), origin='lower')
ax.set_title('Average')

for j,g in enumerate(gammas, start=2):
    ax = plt.subplot(1,cols,j)
    ax.imshow(policies_disc[g].reshape((MAX_CARS+1, MAX_CARS+1)), origin='lower')
    ax.set_title(f"γ={g}")
plt.tight_layout(); plt.show()

# Our code

## 7. Optimized Implementation Using Pre-computed Tensors

This section reimplements the LP formulations using optimizations from the policy/value iteration approach:
- Pre-computed transition probability tensor
- Pre-computed expected rentals lookup tables  
- Pre-computed reward matrix
- MDP class with no gamma dependency (reusable for gamma ladder)

In [ ]:
class MDP:
	def __init__(self, action_range: tuple[int,int], location_capacity: tuple[int,int], 
	             rental_params: tuple[int,int], return_params: tuple[int,int]):
		"""
		Initializes the MDP environment for the car rental problem.
		Note: gamma is NOT a parameter - it's an algorithm parameter, not environment parameter.

		Args:
			action_range: A tuple (min_action, max_action) defining the inclusive range of actions.
			location_capacity: The maximum car capacity for each location, e.g., (20, 20).
			rental_params: The average rental requests (λ) for each location.
			return_params: The average car returns (λ) for each location.
		"""
		self.action_range = action_range
		self.actions = np.arange(self.action_range[0], self.action_range[1] + 1)
		self.location_capacity = location_capacity
		self.rental_params = rental_params
		self.return_params = return_params

		# Pre-compute expensive operations once
		print("Pre-computing transition probability matrix...")
		self.transition_prob_matrix = self.build_transition_prob_matrix()
		print("Pre-computing expected reward matrix...")
		self.expected_reward_matrix = self.build_expected_reward_array()
		print("MDP initialization complete!")

	# 1 ---- Compute transition matrix ----
	@staticmethod
	def compute_single_location_transition_prob(cars_morning: int, cars_evening: int, 
	                                            capacity: int, lambda_rent: float, 
	                                            lambda_return: float) -> float:
		"""Calculates the probability of transitioning from cars_morning to cars_evening at a single location."""
		if cars_evening > capacity or cars_evening < 0:
			return 0.0

		total_prob = 0.0

		for cars_rented in range(cars_morning + 1):
			if cars_rented < cars_morning:
				prob_rentals = poisson.pmf(cars_rented, lambda_rent)
			else:
				prob_rentals = poisson.sf(cars_rented - 1, lambda_rent)

			cars_after_rentals = cars_morning - cars_rented
			returns_needed = cars_evening - cars_after_rentals

			if returns_needed < 0:
				continue

			if cars_evening < capacity:
				prob_returns = poisson.pmf(returns_needed, lambda_return)
			else:
				prob_returns = poisson.sf(returns_needed - 1, lambda_return)

			total_prob += prob_rentals * prob_returns

		return total_prob

	@staticmethod
	def build_single_location_transition_prob_matrix(capacity: int, lambda_rent: float, 
	                                                  lambda_return: float) -> np.ndarray:
		"""Builds the (capacity+1, capacity+1) state transition matrix for a single location."""
		num_states = capacity + 1
		transition_matrix = np.zeros((num_states, num_states))

		for morning_state in range(num_states):
			for evening_state in range(num_states):
				prob = MDP.compute_single_location_transition_prob(
					morning_state, evening_state, capacity, lambda_rent, lambda_return
				)
				transition_matrix[morning_state, evening_state] = prob

		return transition_matrix

	def build_transition_prob_matrix(self) -> np.ndarray:
		"""
		Builds the 4D transition probability tensor P[m1, m2, e1, e2].
		This tensor gives the probability of transitioning from a morning state (m1, m2)
		to an evening state (e1, e2).
		"""
		p_loc1 = MDP.build_single_location_transition_prob_matrix(
			self.location_capacity[0], self.rental_params[0], self.return_params[0]
		)
		p_loc2 = MDP.build_single_location_transition_prob_matrix(
			self.location_capacity[1], self.rental_params[1], self.return_params[1]
		)

		# Combine the matrices into a 4D tensor using an outer product.
		# P[m1, m2, e1, e2] = p_loc1[m1, e1] * p_loc2[m2, e2]
		transition_tensor = np.einsum('ik,jl->ijkl', p_loc1, p_loc2)

		return transition_tensor

	# 2 ---- Compute reward matrix ----
	@staticmethod
	def build_expected_rentals_array(capacity: int, lambda_param: float) -> np.ndarray:
		"""Creates an array for expected rentals."""
		expected_rentals = np.zeros(capacity + 1)

		for available_cars in range(capacity + 1):
			current_expected_rentals = 0.0
			for demand in range(available_cars + 1):
				prob = poisson.pmf(demand, lambda_param)
				current_expected_rentals += demand * prob

			# Whenever demand > available_cars, exactly available_cars cars are rented.
			tail_prob = poisson.sf(available_cars, lambda_param)
			current_expected_rentals += available_cars * tail_prob

			expected_rentals[available_cars] = current_expected_rentals

		return expected_rentals

	def build_expected_reward_array(self) -> np.ndarray:
		"""
		Builds the 3D reward array R[action, c1, c2] using pre-computed lookup tables.
		"""
		cap1, cap2 = self.location_capacity
		num_actions = len(self.actions)

		# Pre-compute the expected rentals for all possible car counts at each location.
		expected_rentals_loc1 = MDP.build_expected_rentals_array(cap1, self.rental_params[0])
		expected_rentals_loc2 = MDP.build_expected_rentals_array(cap2, self.rental_params[1])

		reward_array = np.zeros((num_actions, cap1 + 1, cap2 + 1))

		for a_idx, action in enumerate(self.actions):
			for c1 in range(cap1 + 1):
				for c2 in range(cap2 + 1):
					cars_morn_loc1 = c1 - action
					cars_morn_loc2 = c2 + action

					# If the action is invalid, set the reward to -np.inf.
					if not (0 <= cars_morn_loc1 <= cap1 and 0 <= cars_morn_loc2 <= cap2):
						reward_array[a_idx, c1, c2] = -np.inf
					else:
						# Use the fast lookup tables instead of recalculating
						rentals1 = expected_rentals_loc1[cars_morn_loc1]
						rentals2 = expected_rentals_loc2[cars_morn_loc2]

						revenue = RENT_REWARD * (rentals1 + rentals2)
						cost = MOVE_COST * abs(action)
						reward_array[a_idx, c1, c2] = revenue - cost

		return reward_array

In [ ]:
# Create MDP instance - expensive pre-computation happens here
mdp_opt = MDP(
    action_range=(-A_MAX_MOVE, A_MAX_MOVE),
    location_capacity=(MAX_CARS, MAX_CARS),
    rental_params=(LAMBDA_X1, LAMBDA_X2),
    return_params=(LAMBDA_Y1, LAMBDA_Y2)
)

# Build feasible state-action pairs using pre-computed tensors
pairs_opt = []
R_vec_opt = []
P_dense_rows_opt = []

print(f"Building LP structures from pre-computed tensors...")
for s_idx, (n1, n2) in enumerate(STATES):
    for a in FEAS_ACTIONS[s_idx]:
        # Compute morning state after action
        m1, m2 = n1 - a, n2 + a
        
        # Extract transition probabilities from pre-computed tensor (just array indexing!)
        P_row = mdp_opt.transition_prob_matrix[m1, m2, :, :].flatten()
        
        # Extract reward from pre-computed array (just array indexing!)
        a_idx = a - mdp_opt.action_range[0]  # Convert action to index
        R_sa = mdp_opt.expected_reward_matrix[a_idx, n1, n2]
        
        pairs_opt.append((s_idx, a))
        R_vec_opt.append(R_sa)
        P_dense_rows_opt.append(P_row)

K_opt = len(pairs_opt)
R_vec_opt = np.array(R_vec_opt)
P_dense_opt = np.vstack(P_dense_rows_opt)

print(f"Built {K_opt} state-action pairs")
print(f"P_dense shape: {P_dense_opt.shape}")
print(f"R_vec shape: {R_vec_opt.shape}")

In [ ]:
# Solve average-reward LP using optimized structures
A_eq_opt = np.zeros((NUM_STATES+1, K_opt))
b_eq_opt = np.zeros(NUM_STATES+1)

# Flow balance constraints
for k, (s_idx, a) in enumerate(pairs_opt):
    A_eq_opt[s_idx, k] += 1.0
    A_eq_opt[:NUM_STATES, k] -= P_dense_opt[k, :]

# Normalization constraint
A_eq_opt[NUM_STATES, :] = 1.0
b_eq_opt[NUM_STATES] = 1.0

print("Solving average-reward LP...")
res_opt = linprog(-R_vec_opt, A_eq=A_eq_opt, b_eq=b_eq_opt, bounds=[(0,None)]*K_opt, method='highs')
z_opt = res_opt.x
avg_reward_opt = R_vec_opt @ z_opt
print(f"Optimal average reward per day: {avg_reward_opt:.4f}")

# Extract policy
pi_opt = {s_idx: {} for s_idx in range(NUM_STATES)}
z_state_opt = np.zeros(NUM_STATES)
for k, (s_idx, a) in enumerate(pairs_opt):
    pi_opt[s_idx][a] = z_opt[k]
    z_state_opt[s_idx] += z_opt[k]

avg_policy_actions_opt = np.zeros(NUM_STATES, dtype=int)
for s_idx in range(NUM_STATES):
    if z_state_opt[s_idx] > 0:
        avg_policy_actions_opt[s_idx] = max(pi_opt[s_idx].items(), key=lambda kv: kv[1])[0]
    else:
        feas = FEAS_ACTIONS[s_idx]
        avg_policy_actions_opt[s_idx] = min(feas, key=abs) if feas else 0

plt.imshow(avg_policy_actions_opt.reshape((MAX_CARS+1, MAX_CARS+1)), origin='lower')
plt.title('Average Reward Policy (Optimized)')
plt.xlabel('N2'); plt.ylabel('N1'); plt.colorbar(); plt.show()

In [ ]:
# Solve discounted LP for gamma ladder - reusing the SAME MDP instance!
def solve_discounted_lp_optimized(gamma):
    """Solve discounted LP using pre-computed tensors. No recomputation needed!"""
    A = np.zeros((K_opt, NUM_STATES))
    b = np.zeros(K_opt)
    
    for k, (s_idx, a) in enumerate(pairs_opt):
        # Bellman constraint: v(s) >= r(s,a) + gamma * sum P(s'|s,a) v(s')
        # Rearranged: -v(s) + gamma * sum P(s'|s,a) v(s') <= -r(s,a)
        A[k, :] = gamma * P_dense_opt[k, :]
        A[k, s_idx] -= 1.0
        b[k] = -R_vec_opt[k]
    
    res = linprog(np.ones(NUM_STATES), A_ub=A, b_ub=b, 
                  bounds=[(None,None)]*NUM_STATES, method='highs')
    v = res.x
    
    # Extract policy
    policy = np.zeros(NUM_STATES, dtype=int)
    for s_idx, (n1, n2) in enumerate(STATES):
        best_q, best_a = -1e300, 0
        for a in FEAS_ACTIONS[s_idx]:
            k = next(k for k,(ss,aa) in enumerate(pairs_opt) if ss==s_idx and aa==a)
            q = R_vec_opt[k] + gamma * (P_dense_opt[k,:] @ v)
            if q > best_q:
                best_q, best_a = q, a
        policy[s_idx] = best_a
    return policy

gammas_opt = [0.95, 0.97, 0.98, 0.99, 0.995, 0.997, 0.999]
policies_disc_opt = {}

print("Solving discounted LPs for gamma ladder (reusing same MDP)...")
for g in gammas_opt:
    print(f"  Solving for gamma = {g}...")
    policies_disc_opt[g] = solve_discounted_lp_optimized(g)

# Compare with average-reward policy
rates_opt = {g: np.mean(policies_disc_opt[g]==avg_policy_actions_opt) for g in gammas_opt}
for g in gammas_opt:
    print(f"γ={g}: match rate = {rates_opt[g]*100:.1f}%")

In [ ]:
# Visualization of Blackwell phenomenon with optimized implementation
cols = len(gammas_opt)+1
plt.figure(figsize=(3*cols,5))

ax = plt.subplot(1,cols,1)
ax.imshow(avg_policy_actions_opt.reshape((MAX_CARS+1, MAX_CARS+1)), origin='lower')
ax.set_title('Average (Optimized)')
ax.set_xlabel('N2')
ax.set_ylabel('N1')

for j,g in enumerate(gammas_opt, start=2):
    ax = plt.subplot(1,cols,j)
    ax.imshow(policies_disc_opt[g].reshape((MAX_CARS+1, MAX_CARS+1)), origin='lower')
    ax.set_title(f"γ={g}")
    ax.set_xlabel('N2')
    ax.set_ylabel('N1')

plt.tight_layout()
plt.show()

In [ ]:
# Verify optimized results match original results
print("\n=== Comparison with Original Implementation ===")
print(f"Average reward (original): {avg_reward:.4f}")
print(f"Average reward (optimized): {avg_reward_opt:.4f}")
print(f"Difference: {abs(avg_reward - avg_reward_opt):.6f}")

print("\nAverage-reward policy agreement:")
policy_match = np.mean(avg_policy_actions == avg_policy_actions_opt)
print(f"  {policy_match*100:.1f}% of states have same action")

print("\nDiscounted policy agreement for each gamma:")
for g in gammas:
    if g in gammas_opt:
        match = np.mean(policies_disc[g] == policies_disc_opt[g])
        print(f"  γ={g}: {match*100:.1f}% agreement")

### Key Optimizations

The optimized implementation provides significant speedups through:

1. **Pre-computed Transition Tensor** (4D array): Instead of computing transition probabilities with nested loops over rental/return scenarios for each state-action pair, we compute a single 4D tensor once using `np.einsum` and then just index into it.

2. **Pre-computed Expected Rentals Lookup Tables**: Rather than summing Poisson probabilities for each state-action pair, we compute expected rentals for all possible car counts (0-20) once and use O(1) lookups.

3. **Pre-computed Reward Matrix** (3D array): All rewards computed once during initialization, then just indexed during LP construction.

4. **scipy.stats.poisson**: Uses optimized C implementation instead of Python's `math.factorial()`.

5. **Gamma-independent MDP**: The MDP class doesn't depend on gamma, so we build it once and reuse it for the entire gamma ladder (7 different gamma values), avoiding redundant computation.

**Result**: The expensive pre-computation happens once in `MDP.__init__()`, then LP construction and solving are much faster due to simple array indexing instead of nested Poisson calculations.